In [ ]:
%matplotlib inline



# Compute total displacement from distributed files with distributed post
This example shows how distributed files can be read and post processed
on distributed processes. After remote post processing of total displacement,
results a merged on the local process. In this example, the client is only
connected to the coordinator server. Connections to remote processes are only
done implicitly through the coordinator.


Import dpf module and its examples files



In [ ]:
from ansys.dpf import core as dpf
from ansys.dpf.core import examples
from ansys.dpf.core import operators as ops
from ansys.jupyterhub.manager import spawn_dpf, delete_pod

# Create the template workflow of total displacement



In [ ]:
server1, pod1_name = spawn_dpf()
template_workflow = dpf.Workflow(server=server1)
displacement = ops.result.displacement()
norm = ops.math.norm_fc(displacement)

Add the operators to the template workflow and name its inputs and outputs
Once workflow's inputs and outputs are named, they can be connected later on



In [ ]:
template_workflow.add_operators([displacement, norm])
template_workflow.set_input_name("data_sources", displacement.inputs.data_sources)
template_workflow.set_output_name("out", norm.outputs.fields_container)

# Configure the servers
Start two more dpf servers. Workflows instances will be created on each of those servers to
address each a different result file.
In this example, we will post process an analysis distributed in 2 files,
we will consequently require 2 remote processes



In [ ]:
server2, pod2_name = spawn_dpf()
server3, pod3_name = spawn_dpf()
servers = [server2, server3]

Here we show how we could send files in temporary directory if we were not
in shared memory



In [ ]:
local_files = examples.download_distributed_files()
server_file_paths = [dpf.upload_file_in_tmp_folder(local_files[0], server=server2),
                     dpf.upload_file_in_tmp_folder(local_files[1], server=server3)]

# Send workflows on servers
Here we create new instances on the server by copies of the template workflow
We also connect the data sources to those workflows.



In [ ]:
remote_workflows = []
for i, server in enumerate(servers):
    remote_workflows.append(template_workflow.create_on_other_server(server=server))
    ds = dpf.DataSources(server_file_paths[i])
    remote_workflows[i].connect("data_sources", ds)

# Create a workflow able to merge the results



In [ ]:
workflow = dpf.Workflow()
merge = ops.utility.merge_fields_containers()
workflow.add_operator(merge)
workflow.set_input_name("in0", merge, 0)
workflow.set_input_name("in1", merge, 1)
workflow.set_output_name("merged", merge.outputs.merged_fields_container)

# Connect the workflows together and get the output



In [ ]:
for i, server in enumerate(servers):
    workflow.connect_with(remote_workflows[i], ("out", "in" + str(i)))

fc = workflow.get_output("merged", dpf.types.fields_container)
print(fc)
print(fc[0].min().data)
print(fc[0].max().data)

In [ ]:
delete_pod(pod1_name)
delete_pod(pod2_name)
delete_pod(pod3_name)